# FK Processing (CLI)

A series of steps illustrating how to begin processing infrasonic data utilizing the command line methods of Infrapy

## Building a database for processing

The CLI methods in Infrapy depend on a relational SQL database for pointing to data for processing and storing processing results. In this tutorial, we will build a SQLite database out of SAC files provided in the /test folder, and begin FK processing.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

%matplotlib notebook

In [ ]:
!pisces sac2db sqlite:///test_cli.sqlite ../infrapy-data/cli/*.SAC

As infrapy is an array processing tool, after your sqlite database is created, you will need to update the REFSTA for each array using update_refsta.py

In [ ]:
%run ../scripts/update_refsta.py sqlite:///test_cli.sqlite FSU
%run ../scripts/update_refsta.py sqlite:///test_cli.sqlite HWU
%run ../scripts/update_refsta.py sqlite:///test_cli.sqlite LCM
%run ../scripts/update_refsta.py sqlite:///test_cli.sqlite PSU
%run ../scripts/update_refsta.py sqlite:///test_cli.sqlite WMU

## Updating Configuration File for Processing

An example configuration file is provided in the /test folder. A full description of each section of the configuration file is provided in the documentation.
The configuration file outlines the database you wish to process in, the specific parameters for processing and the station [or network for assocation processing] to use. For simplicity within this tutorial, since we have loaded data from five separate arrays into the database, we will call processing in the same database with five different files.

### Viewing the configuration file

We can use the %load command to view a configuration file within this notebook.

In [ ]:
#%load config_fsu.txt
[database] # required
url = sqlite:///test_cli.sqlite
site = pisces.tables.css3:Site
wfdisc = pisces.tables.css3:Wfdisc
affiliation = pisces.tables.css3:Affiliation

[GeneralParams]
year=2012
dayofyearini=206
dayofyearend=208
station=FSU
channel=EDF
name=test
cpucnt=30


[FKParams]
name=mid band fk test
freqmin=0.5
freqmax=5
beamwinlen=60
beamwinstep=30
backazmin=-180.0
backazmax=180.0
backazstep=1.5
trvelmin=300.0
trvelmax=600.0
trvelstep=2.5
beammethod=bartlett
fkresults=fk_res_fsu
numsources = 1
func_fk = None

[FDetectParams]
detwinlen=1800.0
detthresh=0.99
dsegmin=5
back_az_lim=15
detmethod=fstat
tb_prod=4000
pfkid=0
fkresults=fk_res_fsu
fdresults=fd_res_fsu




[AssocLocParams]
network=YJ
pfdetectid=0
pfkid=0
distmax=10.0
beamwidth=10.0
rangemax=1000.0
clusterthresh=4.0
trimthresh=None
trimthreshscalar=1.0
mindetpop=3
minarraypop=2
duration = 60
name=test
fdtable_1=fd_res_fsu
fdtable_2=fd_res_hwu
fdtable_3=fd_res_lcm
fdtable_4=fd_res_psu
fdtable_5=fd_res_wmu
resultstable = test_assoc_1



In [ ]:
!infrapy run_fk --config_file config_fsu.txt

## Using scripts to output processing results from the DB

### Plot FK Results

In [ ]:
!plot1_rfk.py -h

In [ ]:
%run ../scripts/plot1_rfk.py -d sqlite:///test_cli.sqlite -a FSU -domain time -f 0 -t fk_res_fsu 

### Save FK results to text file

In [ ]:
!print_fk.py -h

In [ ]:
#!print_fk.py -d sqlite:///test.sqlite -a FSU -t fk_res_fsu -f 0 -o fsu_fk.txt

### Print out all FK processing parameters used

In [ ]:
!read_pfk.py -h

In [ ]:
!read_pfk.py -d sqlite:///test_cli.sqlite

### Plot PSD of data

In [ ]:
!plot_psd_array.py -h

In [ ]:
%matplotlib notebook
%run ../scripts/plot_psd_array.py -d sqlite:///test_cli.sqlite -a FSU -S '2012-07-24T08:50:00' -E '2012-07-24T09:20:00' -l 200

## Repeat same processing steps on the other arrays within the network

### Running CLI FK Processing on data from HWU

In [ ]:
!infrapy run_fk --config_file config_hwu.txt

### Running CLI FK Processing on data from LCM

In [ ]:
!infrapy run_fk --config_file config_lcm.txt

### Running CLI FK Processing on data from NOQ

In [ ]:
!infrapy run_fk --config_file config_noq.txt

### Running CLI FK Processing on data from PSU

In [ ]:
!infrapy run_fk --config_file config_psu.txt

### Running CLI FK Processing on data from WMU

In [ ]:
!infrapy run_fk --config_file config_wmu.txt